In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd '/content/gdrive/MyDrive/Colab Notebooks/Modified-IRNet'

/content/gdrive/MyDrive/Colab Notebooks/Modified-IRNet


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4') ##Required for Colab

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
pip install pattern

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 60.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 KB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 114.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 105.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.4/348.4 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 KB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 74.2 MB/s eta 0:00:00
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332722 sha256=d02f79

In [ ]:
import torch
from src import args as arg
from src import utils
from src.models.model import IRNet
from src.rule import semQL

In [ ]:
## Copied from IRNET
def evaluate(args):
    """
    :param args:
    :return:
    """

    grammar = semQL.Grammar()
    sql_data, table_data, val_sql_data,\
    val_table_data= utils.load_dataset(args.dataset, use_small=args.toy)

    model = IRNet(args, grammar)

    if args.cuda: model.cuda()

    print('load pretrained model from %s'% (args.load_model))
    pretrained_model = torch.load(args.load_model,
                                     map_location=lambda storage, loc: storage)
    import copy
    pretrained_modeled = copy.deepcopy(pretrained_model)
    for k in pretrained_model.keys():
        if k not in model.state_dict().keys():
            del pretrained_modeled[k]

    model.load_state_dict(pretrained_modeled)

    model.word_emb = utils.load_word_emb(args.glove_embed_path)

    json_datas, sketch_acc, acc = utils.epoch_acc(model, args.batch_size, val_sql_data, val_table_data,
                           beam_size=args.beam_size)
    print('Sketch Acc: %f, Acc: %f' % (sketch_acc, acc))
    # utils.eval_acc(json_datas, val_sql_data)
    import json
    with open('./predict_lf.json', 'w') as f:
        json.dump(json_datas, f)


In [ ]:
class args:
    dataset = './data'
    glove_embed_path = './data/glove.42B.300d.txt'
    cuda = True
    epoch = 50
    loss_epoch_threshold = 50
    sketch_loss_coefficient = 1.0
    beam_size = 5 ## SS: Different from train which is 1
    seed = 90
    save = './'
    embed_size = 300
    sentence_features = True
    column_pointer = True
    hidden_size = 300
    lr_scheduler = True
    lr_scheduler_gammar = 0.5
    att_vec_size = 300
    toy=False
    model_name = 'modified_irnet'
    lstm = 'lstm'
    load_model = './saved_model/1667898752_0.75_new_loss_fn_50percent_DA/best_model_0.75_new_loss_fn_50percent_DA.model'
    batch_size = 64
    col_embed_size = 300
    action_embed_size = 128
    type_embed_size = 128
    dropout = 0.3
    word_dropout = 0.2
    no_query_vec_to_action_map = False
    readout = 'identity'
    query_vec_to_action_diff_map = False
    column_att = 'affine'
    decode_max_time_step = 40
    save_to = 'model'
    clip_grad = 5.
    max_epoch = -1
    optimizer = 'Adam'
    lr = 0.001

In [ ]:
evaluate(args)

Loading from datasets...
Loading data from ./data/tables.json
Loading data from ./data/train.json
Loading data from ./data/dev.json
Use Column Pointer:  True
load pretrained model from ./saved_model/1667898752_0.75_new_loss_fn_50percent_DA/best_model_0.75_new_loss_fn_50percent_DA.model
Loading word embedding from ./data/glove.42B.300d.txt
./data/glove.42B.300d.txt


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Sketch Acc: 0.816505, Acc: 0.516505
